<a href="https://colab.research.google.com/github/ChinmayBhagwat1/Data-Analyst/blob/main/Car_Price_Prediction_PRJ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#Importing all the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split,GridSearchCV

from sklearn.metrics import r2_score,mean_squared_error

import datetime
Dtime = datetime.datetime.now()

import warnings
warnings.filterwarnings
warnings.filterwarnings('ignore')

In [5]:
#Reading the csv file
df = pd.read_csv("car_price_prediction.csv")


In [6]:
#Starting with the data overview
df.shape

(19237, 18)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19237 entries, 0 to 19236
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ID                19237 non-null  int64 
 1   Price             19237 non-null  int64 
 2   Levy              19237 non-null  object
 3   Manufacturer      19237 non-null  object
 4   Model             19237 non-null  object
 5   Prod. year        19237 non-null  int64 
 6   Category          19237 non-null  object
 7   Leather interior  19237 non-null  object
 8   Fuel type         19237 non-null  object
 9   Engine volume     19237 non-null  object
 10  Mileage           19237 non-null  object
 11  Cylinders         19237 non-null  int64 
 12  Gear box type     19237 non-null  object
 13  Drive wheels      19237 non-null  object
 14  Doors             19237 non-null  object
 15  Wheel             19237 non-null  object
 16  Color             19237 non-null  object
 17  Airbags     

In [8]:
df.describe()

,ID,Price,Prod. year,Cylinders,Airbags
count,1.923700e+04,1.923700e+04,19237.000000,19237.000000,19237.000000
mean,4.557654e+07,1.855593e+04,2010.912824,4.582991,6.582627
std,9.365914e+05,1.905813e+05,5.668673,1.199933,4.320168
min,2.074688e+07,1.000000e+00,1939.000000,1.000000,0.000000
25%,4.569837e+07,5.331000e+03,2009.000000,4.000000,4.000000
50%,4.577231e+07,1.317200e+04,2012.000000,4.000000,6.000000
75%,4.580204e+07,2.207500e+04,2015.000000,4.000000,12.000000
max,4.581665e+07,2.630750e+07,2020.000000,16.000000,16.000000


In [9]:
#Data preprocessing
df.duplicated().sum()
df.drop_duplicates(inplace = True)
df.shape

(18924, 18)

In [10]:
df.isna().sum()

,0
ID,0
Price,0
Levy,0
Manufacturer,0
Model,0
Prod. year,0
Category,0
Leather interior,0
Fuel type,0
Engine volume,0


In [13]:
#Calculating the age of the car
df["Age"] = Dtime.year - df["Prod. year"]
df.head()

,ID,Price,Levy,Manufacturer,Model,Prod. year,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Doors,Wheel,Color,Airbags,Age
0,45654403,13328,1399,LEXUS,RX 450,2010,Jeep,Yes,Hybrid,3.5,186005 km,6,Automatic,4x4,4-May,Left wheel,Silver,12,14
1,44731507,16621,1018,CHEVROLET,Equinox,2011,Jeep,No,Petrol,3,192000 km,6,Tiptronic,4x4,4-May,Left wheel,Black,8,13
2,45774419,8467,-,HONDA,FIT,2006,Hatchback,No,Petrol,1.3,200000 km,4,Variator,Front,4-May,Right-hand drive,Black,2,18
3,45769185,3607,862,FORD,Escape,2011,Jeep,Yes,Hybrid,2.5,168966 km,4,Automatic,4x4,4-May,Left wheel,White,0,13
4,45809263,11726,446,HONDA,FIT,2014,Hatchback,Yes,Petrol,1.3,91901 km,4,Automatic,Front,4-May,Left wheel,Silver,4,10


In [27]:
#Data Cleaning
df["Levy"] = df.Levy.replace({"-":"0"}, inplace = True)
# astype(int)
df["Mileage"] = df["Mileage"].str.replace("km","")
#
df['Engine volume'] = df['Engine volume'].astype(str)
df['Engine volume'] = df['Engine volume'].str.replace('Turbo','')
df['Engine volume'] = df['Engine volume'].astype(float)
#
data = df.drop(['ID','Doors','Prod. year'], axis =1)
data

,Price,Levy,Manufacturer,Model,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Wheel,Color,Airbags,Age
0,13328,None,LEXUS,RX 450,Jeep,Yes,Hybrid,3.5,186005,6,Automatic,4x4,Left wheel,Silver,12,14
1,16621,None,CHEVROLET,Equinox,Jeep,No,Petrol,3.0,192000,6,Tiptronic,4x4,Left wheel,Black,8,13
2,8467,None,HONDA,FIT,Hatchback,No,Petrol,1.3,200000,4,Variator,Front,Right-hand drive,Black,2,18
3,3607,None,FORD,Escape,Jeep,Yes,Hybrid,2.5,168966,4,Automatic,4x4,Left wheel,White,0,13
4,11726,None,HONDA,FIT,Hatchback,Yes,Petrol,1.3,91901,4,Automatic,Front,Left wheel,Silver,4,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19232,8467,None,MERCEDES-BENZ,CLK 200,Coupe,Yes,CNG,2.0,300000,4,Manual,Rear,Left wheel,Silver,5,25
19233,15681,None,HYUNDAI,Sonata,Sedan,Yes,Petrol,2.4,161600,4,Tiptronic,Front,Left wheel,Red,8,13
19234,26108,None,HYUNDAI,Tucson,Jeep,Yes,Diesel,2.0,116365,4,Automatic,Front,Left wheel,Grey,4,14
19235,5331,None,CHEVROLET,Captiva,Jeep,Yes,Diesel,2.0,51258,4,Automatic,Front,Left wheel,Black,4,17


In [29]:
#Checking data
data.head()

,Price,Levy,Manufacturer,Model,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Wheel,Color,Airbags,Age
0,13328,None,LEXUS,RX 450,Jeep,Yes,Hybrid,3.5,186005,6,Automatic,4x4,Left wheel,Silver,12,14
1,16621,None,CHEVROLET,Equinox,Jeep,No,Petrol,3.0,192000,6,Tiptronic,4x4,Left wheel,Black,8,13
2,8467,None,HONDA,FIT,Hatchback,No,Petrol,1.3,200000,4,Variator,Front,Right-hand drive,Black,2,18
3,3607,None,FORD,Escape,Jeep,Yes,Hybrid,2.5,168966,4,Automatic,4x4,Left wheel,White,0,13
4,11726,None,HONDA,FIT,Hatchback,Yes,Petrol,1.3,91901,4,Automatic,Front,Left wheel,Silver,4,10


In [ ]:
#Data Transformation
